In [1]:
import os
import json
import numpy as np
import torch
import result
import pandas as pd
from rnn_model import LSTMNet, GRUNet
from utils import DotDict, Logger, rmse, rmse_tensor, rmse_np, boolean_string, get_dir, get_time, next_dir, model_dir, get_model
from get_dataset import get_time_data, get_stnn_data
from stnn import *
import matplotlib.pyplot as plt
from matplotlib import axis
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
%matplotlib inline

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


# pred 7 time step

## Best Jar

In [2]:
best_jar_3_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'classical_7', 'jar_classical_7'))
jar_result = result.Printer(best_jar_3_folder)
pd.options.display.max_rows = None
jar_df = jar_result.get_df(col=['minrmse', 'minsum', 'min_rmse_epoch', 'activation', 'data_normalize', 'nlayers', 'nhid', 'activation', 'nz', 'validation_length'], increase=False).sort_values('minrmse')
jar_df

,minrmse,minsum,min_rmse_epoch,activation,data_normalize,nlayers,nhid,activation,nz,validation_length
classical-stnn_09-15-10-54-08_9212,578.538,5585.7,69,tanh,d,1,0,tanh,10,7
classical-stnn_09-15-10-54-05_5322,578.959,5480.3,6,tanh,d,2,100,tanh,10,7
classical-stnn_09-15-10-54-05_5629,579.952,5246.18,397,sigmoid,d,1,0,sigmoid,10,7
classical-stnn_09-15-10-54-07_9938,580.097,5594.1,627,sigmoid,d,1,0,sigmoid,10,7
classical-stnn_09-15-10-53-55_8069,580.216,5946.18,3,tanh,d,3,100,tanh,40,7
classical-stnn_09-15-10-54-01_7122,580.244,5684.34,769,sigmoid,d,1,0,sigmoid,40,7
classical-stnn_09-15-10-54-08_9605,580.299,5797.27,1,sigmoid,d,2,100,sigmoid,10,7
classical-stnn_09-15-10-54-07_2616,580.41,6291.1,0,tanh,d,3,100,tanh,10,7
classical-stnn_09-15-10-54-05_9642,580.413,4894.7,553,tanh,d,1,0,tanh,10,7
classical-stnn_09-15-10-54-05_3155,580.481,5741.76,3,tanh,d,4,100,tanh,10,7


In [3]:
result.output_scr_by_dir({"classical": "classical-stnn_09-15-10-54-08_9212"}, "../classical_7/jar_classical_7", minepoch="rmse", write="a")

# Feb

In [4]:
best_feb_3_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'classical_7', 'feb_classical_7'))
feb_result = result.Printer(best_feb_3_folder)
pd.options.display.max_rows = None
feb_df = feb_result.get_df(col=['minrmse', 'minsum', 'min_rmse_epoch', 'activation', 'data_normalize', 'nlayers', 'nhid', 'activation', 'nx'], increase=False).sort_values('minrmse')
feb_df

,minrmse,minsum,min_rmse_epoch,activation,data_normalize,nlayers,nhid,activation,nx
classical-stnn_09-15-10-39-11_5599,891.91,2369.75,87,tanh,d,3,100,tanh,31
classical-stnn_09-15-10-39-11_6877,891.978,1633.04,131,sigmoid,d,2,100,sigmoid,31
classical-stnn_09-15-10-39-10_8921,892.071,1654.26,280,tanh,d,4,100,tanh,31
classical-stnn_09-15-10-39-11_4902,892.073,1632.54,168,sigmoid,d,2,100,sigmoid,31
classical-stnn_09-15-10-39-11_9721,892.122,4627.46,396,tanh,d,4,100,tanh,31
classical-stnn_09-15-10-39-11_7907,892.125,1622.64,182,sigmoid,d,2,100,sigmoid,31
classical-stnn_09-15-10-39-11_5418,892.127,1541.69,441,tanh,d,3,100,tanh,31
classical-stnn_09-15-10-39-09_4503,892.183,1574.19,227,sigmoid,d,2,100,sigmoid,31
classical-stnn_09-15-10-39-11_7038,892.184,2150.03,502,tanh,d,4,100,tanh,31
classical-stnn_09-15-10-39-10_7936,892.201,1623.49,496,tanh,d,3,100,tanh,31


In [ ]:
result.output_scr_by_dir({"classical": "classical-stnn_09-15-10-54-08_9212"}, "../classical_7/jar_classical_7", minepoch="rmse", write="a")

In [5]:
best_mar_7_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'classical_7', 'mar_classical_7'))
mar_result = result.Printer(best_mar_7_folder)
pd.options.display.max_rows = None
mar_df = mar_result.get_df(col=['minrmse', 'minsum', 'min_rmse_epoch', 'activation', 'data_normalize', 'nlayers', 'nhid', 'activation', 'nx', 'validation_length'], increase=False).sort_values('minrmse')
mar_df

,minrmse,minsum,min_rmse_epoch,activation,data_normalize,nlayers,nhid,activation,nx,validation_length
classical-stnn_09-15-11-13-07_3948,72.0574,171.591,73,tanh,d,4,100,tanh,31,7
classical-stnn_09-15-11-13-07_5408,72.1274,170.091,251,sigmoid,d,2,100,sigmoid,31,7
classical-stnn_09-15-11-13-08_0477,72.1519,174.002,6260,sigmoid,d,2,100,sigmoid,31,7
classical-stnn_09-15-11-13-07_8033,72.1521,174.234,5918,sigmoid,d,3,100,sigmoid,31,7
classical-stnn_09-15-11-13-07_9309,72.1521,174.739,3534,tanh,d,2,100,tanh,31,7
classical-stnn_09-15-11-13-07_7760,72.1521,174.11,3582,sigmoid,d,3,100,sigmoid,31,7
classical-stnn_09-15-11-13-07_4418,72.1521,169.607,3912,tanh,d,4,100,tanh,31,7
classical-stnn_09-15-11-13-07_8489,72.1521,174.044,416,sigmoid,d,4,100,sigmoid,31,7
classical-stnn_09-15-11-13-08_0479,72.1521,172.966,3633,tanh,d,2,100,tanh,31,7
classical-stnn_09-15-11-13-07_1557,72.1521,151.342,4735,tanh,d,2,100,tanh,31,7


In [ ]:
result.output_scr_by_dir({"v4": "STNN-I-stnn_09-11-18-31-28_1136"}, "../v4-output/mar_v4", minepoch="rmse", write='a')